In [1]:
import pandas as pd
import numpy as np

In [2]:
dateparse=lambda x: pd.datetime.strptime(x, '%d.%m.%Y %H:%M:%S.000')
filename='GBPJPY_Candlestick_30_m_BID_14.08.2017-19.08.2017.csv'
df=pd.read_csv(filename, index_col=0, parse_dates=True, date_parser=dateparse)
df.tail()

,Open,High,Low,Close,Volume
Local time,,,,,
2017-08-19 21:30:00,140.55,140.55,140.55,140.55,0.0
2017-08-19 22:00:00,140.55,140.55,140.55,140.55,0.0
2017-08-19 22:30:00,140.55,140.55,140.55,140.55,0.0
2017-08-19 23:00:00,140.55,140.55,140.55,140.55,0.0
2017-08-19 23:30:00,140.55,140.55,140.55,140.55,0.0


In [3]:
import matplotlib.pyplot as plt
df['Close'].plot(figsize=(10,5), linewidth=0.5)

In [4]:
df['Close_Diff']=df['Close'] - df['Close'].shift(1)
df['Close_Diff'].plot(figsize=(10,5), color='r')

df['Close_Diff2']=df['Close_Diff'] / (df['Close_Diff'].max()-df['Close_Diff'].min())
df['Close_Diff2'].plot(figsize=(10,5))


In [5]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MINST data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [6]:
# Parameters
learning_rate = 0.0001#0.001
training_iters = 500000#100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [7]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `bidirectional_rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = BiRNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [11]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
#        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x, batch_y = df['Close_Diff2'].next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = df['Close_Diff2'].test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = df['Close_Diff2'].test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

AttributeError: 'Series' object has no attribute 'next_batch'